In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import math as ma
from astropy.io import fits

In [2]:
df = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/observation_id/observation_id.csv", header=0, engine='python')
df['date'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z").date())
df['date_time'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z"))
df['partition'] = pd.factorize(df['date'])[0] + 1
df = df[df['partition'] == 1].reset_index(drop=True)

---

#### Checking the results normal vs faster algorithm

In [3]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    #with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/3/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])
    
    # Create DataFrame
    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1] * data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0] * data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp['time_index'].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp['time_index'].map(lambda t: int(time_blocks[t]))
    #df_temp['timeblock'] = df_temp.timeblock.apply(lambda x: np.where(transform_tb == x)[0].tolist())
    #df_temp = df_temp.explode('timeblock').reset_index(drop=True)

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop = True)


In [4]:
df_sel_rfi_slow = pd.DataFrame()
for o in range(1):
    #aa = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/both_side_selected_outliers_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    aa = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/observation_data/both_side_selected_outliers_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    aa.columns = ['xx', 'yy', 'obs_id', 'timeblocks', 'tile', 'freqs', 'z_score', 'polarization']
    aa = aa[~aa['z_score'].isna()].reset_index(drop=True)
    aa['z_score'] = aa['z_score'].astype(float)
    aa['floor_z_score'] = aa['z_score'].apply(lambda x: ma.floor(x))
    df_sel_rfi_slow = pd.concat([df_sel_rfi_slow, aa]).reset_index(drop = True)

In [ ]:
np.unique(df_sel_rfi_fast['obs_id'])

In [5]:
df_sel_rfi_fast[df_sel_rfi_fast['obs_id'] == 1095450824].head(10)

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
1146880,NaN,NaN,NaN,NaN,1095450824,14,0,0,0
1146881,NaN,NaN,NaN,NaN,1095450824,14,1,0,0
1146882,NaN,NaN,NaN,NaN,1095450824,14,2,0,0
1146883,NaN,NaN,NaN,NaN,1095450824,14,3,0,0
1146884,NaN,NaN,NaN,NaN,1095450824,14,4,0,0
1146885,NaN,NaN,NaN,NaN,1095450824,14,5,0,0
1146886,NaN,NaN,NaN,NaN,1095450824,14,6,0,0
1146887,NaN,NaN,NaN,NaN,1095450824,14,7,0,0
1146888,NaN,NaN,NaN,NaN,1095450824,14,8,0,0
1146889,NaN,NaN,NaN,NaN,1095450824,14,9,0,0


In [6]:
df_sel_rfi_slow[(df_sel_rfi_slow['obs_id'] == 1095450824) & (df_sel_rfi_slow['freqs'] == 0) & (df_sel_rfi_slow['tile'] == 0) & (df_sel_rfi_slow['timeblocks'] == 0)]

,xx,yy,obs_id,timeblocks,tile,freqs,z_score,polarization,floor_z_score


---

#### Min max comparison

In [7]:
df_stat_slow = pd.DataFrame()
for o in range(1):
    #oo = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/both_side_outliers_statistics_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    oo = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/observation_data/both_side_outliers_statistics_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    oo.columns = ['tile', 'freqs', 'polarization', 'skewness', 'max', 'min', 'mean', 'win_mean', 'std', 'win_std', 'gamma_final', 'number_of_outliers']
    oo['grid'] = o
    df_stat_slow = pd.concat([df_stat_slow, oo]).reset_index(drop = True)

In [8]:
df_stat_fast = pd.DataFrame()
for o in range(1):
    #oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/outlier_statistics_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/3/outlier_statistics_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo['grid'] = o
    df_stat_fast = pd.concat([df_stat_fast, oo]).reset_index(drop = True)

In [9]:
df_gamma_fast = pd.DataFrame()
for o in range(1):
    #oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/final_gamma_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/3/final_gamma_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo['grid'] = o
    df_gamma_fast = pd.concat([df_gamma_fast, oo]).reset_index(drop = True)

In [15]:
df_stat_slow[(df_stat_slow['tile'] == 6) & (df_stat_slow['freqs'] == 100)]

,tile,freqs,polarization,skewness,max,min,mean,win_mean,std,win_std,gamma_final,number_of_outliers,grid
6646,6,100,XX,0.403695,2.491822,1.383309,1.844994,1.839580,0.251907,0.218596,0.215,224,0
6647,6,100,YY,0.531858,2.276477,1.155753,1.636792,1.627466,0.197578,0.157379,0.215,224,0


In [16]:
df_stat_fast[(df_stat_fast['antenna'] == 6) & (df_stat_fast['frequency'] == 100) & (df_stat_fast['polarization'].isin([0,3]))]

,antenna,frequency,polarization,ori_mean,ori_std,ori_max,ori_min,ori_skew,winsorized_mean,winsorized_std,winsorized_skew,grid
15760,6,100,0,1.844994,0.251907,2.491822,1.383309,0.403695,1.841296,0.221921,0.271950,0
15763,6,100,3,1.636792,0.197578,2.276477,1.155753,0.531858,1.630131,0.161987,0.220994,0


In [14]:
df_gamma_fast[(df_gamma_fast['antenna'] == 6) & (df_gamma_fast['frequency'] == 100) & (df_stat_fast['polarization'].isin([0,3]))]

,antenna,frequency,polarization,final_gamma,grid
15760,6,100,0,0.19697,0
15763,6,100,3,0.19697,0


---

In [ ]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])
    
    # Create DataFrame
    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1] * data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0] * data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp['time_index'].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp['time_index'].map(lambda t: int(time_blocks[t]))
    #df_temp['timeblock'] = df_temp.timeblock.apply(lambda x: np.where(transform_tb == x)[0].tolist())
    #df_temp = df_temp.explode('timeblock').reset_index(drop=True)

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop = True)


In [ ]:
df_sel_rfi_fast[df_sel_rfi_fast['time_index'] == 0]

In [ ]:
f = fits.open(f"/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/calibration_solutions/20250712_observation_data_8s_before_with_cotter_time_corrections_without_flags/uvfits/calibrated_fits/hyperdrive_solutions_1095450704.fits")

In [ ]:
f['SOLUTIONS'].data[:, :, :, ::2]

In [ ]:
g = fits.open(f"/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/calibration_solutions/20240831_multiple_simulation_higher_thermal_noise_sn3_8s/uvfits/calibrated_fits/hyperdrive_solutions_1095450704.fits")

In [ ]:
g['SOLUTIONS'].data[:, :, :, ::2]

---

#### Importing h5 file for flagging

In [17]:
with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/3/outliers_location_day_1_grid_%s_integration_8_real.h5' %(0), "r") as f:
    data = f["outliers_mask"][:]
    obs_id = f["obs_id"][:]
    time_blocks = f["time_blocks"][:]

In [22]:
flat_data = data.reshape(-1, data.shape[-1])
    
# Create DataFrame
df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1] * data.shape[2])
df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0] * data.shape[1])
df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
df_temp["obs_id"] = df_temp['time_index'].map(lambda t: obs_id[t])
df_temp["timeblock"] = df_temp['time_index'].map(lambda t: int(time_blocks[t]))

In [27]:
df_temp[(df_temp['XX'] == True) | (df_temp['XY'] == True) | (df_temp['YX'] == True) | (df_temp['YY'] == True)].reset_index(drop=True)

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
0,False,False,False,True,1095450704,1,21,0,1
1,False,False,True,False,1095450704,1,171,0,1
2,False,True,False,False,1095450704,1,211,0,1
3,True,False,False,False,1095450704,1,227,0,1
4,False,False,False,True,1095450704,1,283,0,1
...,...,...,...,...,...,...,...,...,...
561870,True,False,False,False,1095452528,222,434,127,12
561871,False,False,True,False,1095452528,222,454,127,12
561872,False,False,True,False,1095452528,222,503,127,12
561873,False,False,True,False,1095452528,222,621,127,12
